In [0]:
!pip install geopandas

In [0]:
!apt install libspatialindex-dev
!pip install rtree

In [0]:
import geopandas as gpd
import pandas as pd

In [0]:
!sudo apt-get update && apt-get install -y libspatialindex-dev

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd = '/content/drive/My Drive/AKS_data/TestSolve/'
target_crs = 'epsg:32748'

In [0]:
plants = gpd.read_file(cd + 'Tree_plantations.shp')
aoi = gpd.read_file(cd + 'OKI.shp')
val_ps = gpd.read_file(cd + 'validation_1000samples_interpreted.shp')

In [0]:
plants = gpd.clip(plants.drop(
    columns=filter(lambda x: x not in ['spec_simp', 'geometry'], plants.columns)), aoi)

In [0]:
c_ref = val_ps.drop(columns=['geometry']).set_index('Class').T.to_dict('list')
for k in c_ref:
  c_ref[k] = c_ref[k][0]

In [0]:
c_ref

{'Clearing': 6,
 'Fruit': 4,
 'No plantation': 0,
 'Oil palm': 1,
 'Other': 5,
 'Rubber': 3,
 'Wood fiber': 2}

In [0]:
plants['Class_ID'] = plants.apply(lambda row: c_ref[row.spec_simp.replace(' mix', '')\
  .replace('Unknown', 'Other').replace(' / timber', '').replace('Recently cleared', 'Clearing')], axis=1)

In [0]:
plants = plants.to_crs({'init': target_crs})
plants['area_ha'] = plants['geometry'].area / 10 ** 4

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
plants.head()

,spec_simp,geometry,Class_ID,area_ha
253998,Recently cleared,"MULTIPOLYGON (((544529.135 9728484.455, 544522...",6,28.239972
254046,Recently cleared,"POLYGON ((555764.289 9729690.068, 555656.231 9...",6,1914.272136
254056,Recently cleared,"POLYGON ((548929.093 9728997.070, 548977.425 9...",6,746.826608
254083,Recently cleared,"POLYGON ((552221.491 9725532.957, 552345.321 9...",6,109.420869
254089,Recently cleared,"POLYGON ((556667.664 9725295.284, 556681.777 9...",6,355.935900


In [0]:
aoi = aoi.to_crs({'init': target_crs})
aoi['area'] = aoi['geometry'].area / 10 ** 4

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
aoi.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,area
0,IDN,Indonesia,IDN.31_1,Sumatera Selatan,None,IDN.31.9_1,Ogan Komering Ilir,None,None,Kabupaten,Regency,1602,ID.SL.OH,"MULTIPOLYGON (((460218.207 9595942.624, 460071...",1.793375e+06


In [0]:
aoi_area = aoi['area'][0]

In [0]:
area_by_classes = plants.groupby('Class_ID').sum()
no_p = [aoi_area - sum(area_by_classes['area_ha'])]
row_df = pd.DataFrame([no_p], columns=['area_ha'])
area_by_classes = pd.concat([row_df, area_by_classes], ignore_index=True)
area_by_classes['proportion'] = area_by_classes['area_ha'] / aoi_area

In [0]:
from tqdm import tqdm

cols = sorted(c_ref, key=lambda x: c_ref[x])
val_ps = val_ps.to_crs({'init': target_crs})
c_ref_inv = dict([(cid, name) for name, cid in c_ref.items()])
user_mistake = pd.DataFrame(0.0, columns=[*cols, 'summ', 'mistake'], index=cols)
user_mistake

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,No plantation,Oil palm,Wood fiber,Rubber,Fruit,Other,Clearing,summ,mistake
No plantation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oil palm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wood fiber,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rubber,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fruit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Other,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Clearing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
for index, point in tqdm(val_ps.iterrows()):
    for i, poly in plants.iterrows():
        if point.geometry.within(poly.geometry):
          user_mistake[point.Class][c_ref_inv[poly.Class_ID]] += 1
          break
    else:
      user_mistake[point.Class]['No plantation'] += 1



0it [00:00, ?it/s]

3it [00:00, 27.42it/s]

6it [00:00, 26.42it/s]

11it [00:00, 29.20it/s]

14it [00:00, 27.80it/s]

18it [00:00, 28.10it/s]

21it [00:00, 28.44it/s]

24it [00:00, 27.71it/s]

27it [00:00, 27.57it/s]

30it [00:01, 27.43it/s]

33it [00:01, 25.40it/s]

36it [00:01, 24.04it/s]

41it [00:01, 28.34it/s]

45it [00:01, 26.34it/s]

48it [00:01, 26.80it/s]

51it [00:01, 27.02it/s]

55it [00:01, 27.75it/s]

58it [00:02, 26.77it/s]

61it [00:02, 26.07it/s]

64it [00:02, 24.69it/s]

67it [00:02, 23.72it/s]

72it [00:02, 27.22it/s]

76it [00:02, 29.26it/s]

80it [00:02, 29.77it/s]

84it [00:02, 29.16it/s]

88it [00:03, 28.22it/s]

91it [00:03, 28.00it/s]

94it [00:03, 26.25it/s]

98it [00:03, 28.46it/s]

102it [00:03, 28.88it/s]

106it [00:03, 29.29it/s]

111it [00:03, 31.36it/s]

115it [00:04, 32.51it/s]

121it [00:04, 36.26it/s]

125it [00:04, 36.19it/s]

129it [00:04, 35.77it/s]

133it [00:04, 30.67it/s]

137it [00:04, 32.28it/s]

141it [00:04, 30.50it/s]

145it [00:04, 29.19i

Ошибка пользователя

In [0]:
for idx, row in user_mistake.iterrows():
  row.summ = sum(row[:-2])
  row.mistake = (row.summ - row[idx]) / row.summ
  print(f'{idx}: {(row.mistake * 100):.2f}%')
user_mistake

No plantation: 12.59%
Oil palm: 24.04%
Wood fiber: 28.25%
Rubber: 41.32%
Fruit: 71.43%
Other: 77.78%
Clearing: 81.48%


,No plantation,Oil palm,Wood fiber,Rubber,Fruit,Other,Clearing,summ,mistake
No plantation,479.0,5.0,4.0,19.0,2.0,12.0,27.0,548.0,0.125912
Oil palm,7.0,79.0,0.0,0.0,1.0,2.0,15.0,104.0,0.240385
Wood fiber,25.0,0.0,127.0,0.0,0.0,1.0,24.0,177.0,0.282486
Rubber,22.0,6.0,0.0,71.0,3.0,16.0,3.0,121.0,0.413223
Fruit,2.0,1.0,0.0,0.0,4.0,7.0,0.0,14.0,0.714286
Other,5.0,1.0,0.0,1.0,0.0,2.0,0.0,9.0,0.777778
Clearing,3.0,12.0,4.0,2.0,1.0,0.0,5.0,27.0,0.814815


Ошибка производителя и общая ошибка

In [0]:
 pd.DataFrame(list(area_by_classes['proportion']), cols)[0]

No plantation    0.543950
Oil palm         0.099045
Wood fiber       0.174215
Rubber           0.130523
Fruit            0.008429
Other            0.007166
Clearing         0.036672
Name: 0, dtype: float64

In [0]:
sum_acc = 0
producer_mistake = pd.DataFrame(0.0, columns=[*cols], index=cols)
for col in cols:
  producer_mistake[col] = user_mistake[col] / user_mistake['summ'] * pd.DataFrame(list(area_by_classes['proportion']), cols)[0]
producer_mistake
sum_acc = sum([producer_mistake[i][i] for i in cols])
print(f'Общая точность: {sum_acc * 100:.2f}%\nОбщая ошибка: {(1 - sum_acc) * 100:.2f}%')
for col in cols:
  summ = sum(producer_mistake[col])
  print(f'{col}: {(summ - producer_mistake[col][col]) / summ * 100:.2f}%')

Общая точность: 76.31%
Общая ошибка: 23.69%
No plantation: 11.91%
Oil palm: 27.91%
Wood fiber: 7.00%
Rubber: 22.61%
Fruit: 75.77%
Other: 95.79%
Clearing: 90.91%
